# COSC401 Assignment 2
## Andrew French 
### ID: 11147452

Below is all of the libraries and modules needed for the entire program being imported. 
Make sure to run this step before running any of the feature code.

In [6]:
import numpy as np
import pandas as pd
import torch
import random

Below is all of the variables set that are global to the program.

In [2]:
np.random.seed(19680801)
plt.rcParams['figure.figsize'] = (10,5)

## Part 1 - Gradient-based Learning with Tensors

In [43]:
DATA_SIZE = 1000
RANDOM_STD_DEV = 0.01

In [47]:
def noiseFunction():
    return random.gauss(1, RANDOM_STD_DEV)

def ran_int():
    return random.randint(1, 100)

def inputFunc1(x1, x2, x4):
    return x1 + x2 - x4 + noiseFunction()

def inputFunc2(x2, x3):
    return x2 * 3 + x3 + noiseFunction()

def inputFunc3(x2, x4):
    return x2 + x4 + 2 + noiseFunction()

def inputFunc4(x1, x3):
    return (x1 - x3) * 2 + noiseFunction()

def inputFunc5(x1, x4):
    return (x4 + x1) - 6 + noiseFunction()

def modelFunc(x, w, b):
    """Performs matrix multiplication..."""
    return x @ w.t() + b

def mse(t1, t2):
    """Mean Squared Error loss function"""
    diff = t1 - t2
    return torch.sum(diff * diff) / diff.numel()
    

In [48]:
input_data_list = []
target_data_list = []

for i in range(DATA_SIZE):
    x1, x2, x3, x4 = ran_int(), ran_int(), ran_int(), ran_int()
    input_data_list.append([x1, x2, x3, x4])
    target_data_list.append([inputFunc1(x1, x2, x4), inputFunc2(x2, x3), inputFunc3(x2, x4), inputFunc4(x1, x3), 
                      inputFunc5(x1, x4)])
    
input_data = np.array(input_data_list, dtype='float32')
target_data = np.array(target_data_list, dtype='float32')

#Convert to tensors
inputs = torch.from_numpy(input_data)
targets = torch.from_numpy(target_data)

In [50]:
# Create random Tensors for weights; setting requires_grad=True means that we
# want to compute gradients for these Tensors during the backward pass.
w = torch.randn(5, 4, requires_grad=True)
b = torch.randn(5, requires_grad=True)

preds = modelFunc(inputs, w, b)
loss = mse(preds, targets)

learning_rate = 1e-4
count = 0
while count < 1000000 and loss.item() > 0.04:
    preds = modelFunc(inputs, w, b)
    loss = mse(preds, targets)
    loss.backward()
    with torch.no_grad():
        w -= learning_rate * w.grad
        b -= learning_rate * b.grad
        w.grad.zero_()
        b.grad.zero_()
    if count % 10000 == 0:
        print(f"Iteration: {count}, Loss {loss.item()}")
    count += 1
    


Iteration: 0, Loss 24602.76171875
Iteration: 10000, Loss 0.7094482779502869
Iteration: 20000, Loss 0.6653133034706116
Iteration: 30000, Loss 0.6239269375801086
Iteration: 40000, Loss 0.5851123332977295
Iteration: 50000, Loss 0.5487135648727417
Iteration: 60000, Loss 0.5145819783210754
Iteration: 70000, Loss 0.4825710952281952
Iteration: 80000, Loss 0.4525541067123413
Iteration: 90000, Loss 0.42440658807754517
Iteration: 100000, Loss 0.3980015218257904
Iteration: 110000, Loss 0.3732510209083557
Iteration: 120000, Loss 0.3500339090824127
Iteration: 130000, Loss 0.3282581865787506
Iteration: 140000, Loss 0.3078409731388092
Iteration: 150000, Loss 0.2886984348297119
Iteration: 160000, Loss 0.2707436680793762
Iteration: 170000, Loss 0.2539035677909851
Iteration: 180000, Loss 0.23811420798301697
Iteration: 190000, Loss 0.2233061045408249
Iteration: 200000, Loss 0.2094200700521469
Iteration: 210000, Loss 0.19639381766319275
Iteration: 220000, Loss 0.18419933319091797
Iteration: 230000, Loss 0

## Part 2 - Transfer Learning

In [1]:
# License: BSD
# Author: Sasank Chilamkurthy

from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

plt.ion()   # interactive mode

AttributeError: module 'tensorflow' has no attribute 'placeholder'

## Part 3 - Small Research/Thinking Activity - Cost-Sensitive Learning

Suppose you are given a k-by-k cost matrix C for a classification task which
has k classes. The element Ci,j is the cost of classifying an instance of class j
as class i. You want to train a classifier that minimises the expected cost of
predictions. Do the following:
1. Write a pseudocode or Python code for a loss function that, once optimised
over all examples, achieves minimum expected cost of prediction.
2. Assuming that a learning algorithm which minimises the classification
error is given to you (and that you cannot supply your own loss function
to the algorithm), think and write about a different way of achieving
minimum expected cost of prediction.
Additional notes or requirements:
• Consider cases with k ≥ 2 and briefly discuss if the two methods are
scalable.
• You do not have to use tensors in this questions.
• If you choose to write in Python, you do not have to run your program
on any particular input. You can also use “pseudo-Python” if you wish.

### Part 3.2 - Achieving minimym expected cost of prediction
Idea: if you have a classifer with lots of data and one without, reduce the size of the data to match the smaller one.